In [2]:
import os
import shutil
import json
import glob
import csv
import threading

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mp 
import networkx as nx 

from manager  import RunDict, ResultDict, ItemDict
from analysis import utils
from typing   import List, Dict

%load_ext autoreload
%autoreload 2

In [3]:
from analysis import experiment

INFRA   = "gcp" 

# MINOR
PREFIX  = "2024-07-21-13-08-04"

# MAJOR
PREFIX  = "2024-08-02-13-58-29"

# LEMON
# PREFIX  = "2024-08-04-13-01-35"

DIRPATH = f"analysis/data/treefinder-{INFRA}-{PREFIX}"
DIRPATH = os.path.join(os.getcwd(), DIRPATH)

if not os.path.isdir(DIRPATH):
    raise RuntimeError(f"Not a directory: {DIRPATH}")

DIRPATH

#if EXPERIMENT: 
#    del EXPERIMENT

EXPERIMENT = experiment.Experiment(DIRPATH)

PROC[0]
PROC[1]
PROC[2]
PROC[3]
PARSED RESULT[mkpoGMqvTGDUTdozyPkI] from RUN[BEST-p90-W_5]
PARSED RESULT[oMvGQbjcNPTXZtCCNMnQ] from RUN[BEST-REBUILD-1-p50-W_5]
PARSED RESULT[XcfOFtNfLbbhpUMYlRDT] from RUN[BEST-REBUILD-2-heuristic-W_5]
PARSED RESULT[KCogBJCyuAPEHoYAcGAY] from RUN[BEST-p90-W_5]
PARSED RESULT[NwLHZmPETxoslpAscYxF] from RUN[BEST-p90-W_5]
PARSED RESULT[LaepODJCUNiTMoEzySkC] from RUN[BEST-p90-W_5]
PARSED RESULT[JiAufwqTPcaNVfWavnRF] from RUN[RAND-NONE-W_5]
PARSED RESULT[RLqfFLubegaZyyNlkJQi] from RUN[BEST-p90-W_5]
PARSED RESULT[eQnbhudnFbPTFouBcsjj] from RUN[BEST-REBUILD-2-heuristic-W_5]
PARSED RESULT[eIZBhQCSrhXXiqTnbpkN] from RUN[BEST-REBUILD-1-p50-W_5]
PARSED RESULT[brALPYrVUgNHZPGVTxjd] from RUN[BEST-p90-W_5]
PARSED RESULT[LYwUWGlShXSjNPNVUQnb] from RUN[BEST-p90-W_5]
PARSED RESULT[FaWKtFYDZOMMZrDTaRzD] from RUN[RAND-NONE-W_5]
PARSED RESULT[zqsAdFZEdbkfEtVLiHOV] from RUN[BEST-REBUILD-2-heuristic-W_5]
PARSED RESULT[gYlEuuBJkOzirsnUDPis] from RUN[BEST-REBUILD-1-p50-W_5]
PAR

In [ ]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS       = args.Args(w=28, h=16, f=16, nf=18, tf=24, s=2100)
core.ARGS2      = args.Args(w=28, h=16, f=16, nf=18, tf=24, s=2100, factor=0.8, tbfont=10)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join(DIRPATH, "plot")
utils.mkdir(dir)
    
core.plotAll(dir)

In [ ]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS       = args.Args(w=28, h=16, f=16, nf=18, tf=24, s=2100)
core.ARGS2      = args.Args(w=28, h=16, f=16, nf=18, tf=24, s=2100, factor=0.8, tbfont=10)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join(DIRPATH, "test")
if not os.path.isdir(dir):
    os.mkdir(dir)
    
core.plotTarget(name="BEST", key="heuristic", dir=dir)

In [79]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
# core.ARGS       = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=2100)
# core.ARGS2      = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=1900, factor=1.0, tbfont=13)
core.ARGS2      = args.Args(w=12, h=12, f=20, nf=32, tf=35, s=1900, factor=1.0, tbfont=13)

matplotlib.use('agg')
plt.close("all")

# dir = os.path.join("/home/duclos/Downloads", "test")

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/lemon-experiment")
if not os.path.isdir(dir):
    os.mkdir(dir) 

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/lemon-experiment/a-runs")
if not os.path.isdir(dir):
    os.mkdir(dir) 
    
file="LEMON-AxRUNS.png"
ARR = [
    {"key": "NONE", "name": "LEMON-A-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-A-2",            "root": "" },
    {"key": "NONE", "name": "LEMON-A-3",            "root": "" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
# core.plotEvalComparison(run=runs[0], runs=runs[1:], rdir=dir, title="LEMON-A RUNS", file=file) 
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 


dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/lemon-experiment/b-runs")
if not os.path.isdir(dir):
    os.mkdir(dir)

file="LEMON-BxRUNS.png"
ARR = [
    {"key": "NONE", "name": "LEMON-B-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-B-2",            "root": "" },
    {"key": "NONE", "name": "LEMON-B-3",            "root": "" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
# core.plotEvalComparison(run=runs[0], runs=runs[1:], rdir=dir, title="LEMON-B RUNS", file=file) 
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/lemon-experiment/c-runs")
if not os.path.isdir(dir):
    os.mkdir(dir)

file="LEMON-CxRUNS.png"
ARR = [
    {"key": "NONE", "name": "LEMON-C-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-C-2",            "root": "" },
    {"key": "NONE", "name": "LEMON-C-3",            "root": "" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
# core.plotEvalComparison(run=runs[0], runs=runs[1:], rdir=dir, title="LEMON-C RUNS", file=file)
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True)  

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/lemon-experiment/abc-runs")
if not os.path.isdir(dir):
    os.mkdir(dir)

file="LEMONxOthers.png"
ARR = [
    {"key": "NONE", "name": "LEMON-A-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-B-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-C-1",            "root": "" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
# core.plotEvalComparison(run=runs[0], runs=runs[1:], rdir=dir, title="LEMON PARAMETRIZATION", file=file) 
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True)  

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/lemon-experiment/stress-runs")
if not os.path.isdir(dir):
    os.mkdir(dir)

file="LEMON-STRESSxOthers.png"
ARR = [
    {"key": "NONE", "name": "LEMON-A-STRESS-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-B-STRESS-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-C-STRESS-1",            "root": "" },
] 

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
# core.plotEvalComparison(run=runs[0], runs=runs[1:], rdir=dir, title="LEMON STRESS PARAMETRIZATION", file=file)
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True)  

HERE
PLOTTED: LEMON-A-1-W_1 - table
PLOTTED: LEMON-A-1-W_1 - cdf
PLOTTED: LEMON-A-1-W_1 - p90
PLOTTED: LEMON-A-1-W_1 - p50
TREE LEMON-A-1-W_1 | YLIM 530.5994145549353
PLOTTED: LEMON-A-1-W_1 - stddev
HERE
PLOTTED: LEMON-A-2-W_1 - table
PLOTTED: LEMON-A-2-W_1 - cdf
PLOTTED: LEMON-A-2-W_1 - p90
PLOTTED: LEMON-A-2-W_1 - p50
TREE LEMON-A-2-W_1 | YLIM 1889.9578333149236
PLOTTED: LEMON-A-2-W_1 - stddev
HERE
PLOTTED: LEMON-A-3-W_1 - table
PLOTTED: LEMON-A-3-W_1 - cdf
PLOTTED: LEMON-A-3-W_1 - p90
PLOTTED: LEMON-A-3-W_1 - p50
TREE LEMON-A-3-W_1 | YLIM 491.4955231118591
PLOTTED: LEMON-A-3-W_1 - stddev
HERE
PLOTTED: LEMON-B-1-W_1 - table
PLOTTED: LEMON-B-1-W_1 - cdf
PLOTTED: LEMON-B-1-W_1 - p90
PLOTTED: LEMON-B-1-W_1 - p50
TREE LEMON-B-1-W_1 | YLIM 215.61821325073632
PLOTTED: LEMON-B-1-W_1 - stddev
HERE
PLOTTED: LEMON-B-2-W_1 - table
PLOTTED: LEMON-B-2-W_1 - cdf
PLOTTED: LEMON-B-2-W_1 - p90
PLOTTED: LEMON-B-2-W_1 - p50
TREE LEMON-B-2-W_1 | YLIM 805.3438055828765
PLOTTED: LEMON-B-2-W_1 - stddev
HER

In [80]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
# core.ARGS       = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=2100)
# core.ARGS2      = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=1900, factor=1.0, tbfont=13)
core.ARGS2      = args.Args(w=12, h=12, f=20, nf=32, tf=35, s=1900, factor=1.0, tbfont=13)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/lemon-experiment/abc-runs")
if not os.path.isdir(dir):
    os.mkdir(dir)

file="LEMONxOthers.png"
ARR = [
    {"key": "NONE", "name": "LEMON-A-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-B-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-C-1",            "root": "" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
# core.plotEvalComparison(run=runs[0], runs=runs[1:], rdir=dir, title="LEMON PARAMETRIZATION", file=file) 
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True)  

PLOTTED: LEMON-A-1-W_1 - table
HERE
PLOTTED: LEMON-A-1-W_1 - cdf
PLOTTED: LEMON-A-1-W_1 - p90
PLOTTED: LEMON-A-1-W_1 - p50
TREE LEMON-A-1-W_1 | YLIM 530.5994145549353
PLOTTED: LEMON-A-1-W_1 - stddev
PLOTTED: LEMON-B-1-W_1 - table
HERE
PLOTTED: LEMON-B-1-W_1 - cdf
PLOTTED: LEMON-B-1-W_1 - p90
PLOTTED: LEMON-B-1-W_1 - p50
TREE LEMON-B-1-W_1 | YLIM 215.61821325073632
PLOTTED: LEMON-B-1-W_1 - stddev
PLOTTED: LEMON-C-1-W_1 - table
HERE
PLOTTED: LEMON-C-1-W_1 - cdf
PLOTTED: LEMON-C-1-W_1 - p90
PLOTTED: LEMON-C-1-W_1 - p50
TREE LEMON-C-1-W_1 | YLIM 288.2040291410063
PLOTTED: LEMON-C-1-W_1 - stddev


In [82]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS2      = args.Args(w=12, h=12, f=20, nf=32, tf=35, s=1900, factor=1.0, tbfont=13)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/direct")
if not os.path.isdir(dir):
    os.mkdir(dir)

ARR = [
    {"key": "NONE", "name": "LEMON-A-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-B-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-C-1",            "root": "" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalIterationsComparisonFINAL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

core.plotEvalIterationsComparisonFINAL_ALL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

PLOTTED: DIRECT LEMON-A-1-W_1 - cdf
PLOTTED: DIRECT LEMON-A-1-W_1 - tsp
TREE LEMON-A-1-W_1 | YLIM 312.000996905058
PLOTTED: DIRECT LEMON-B-1-W_1 - cdf
PLOTTED: DIRECT LEMON-B-1-W_1 - tsp
TREE LEMON-B-1-W_1 | YLIM 147.02367047652564
PLOTTED: DIRECT LEMON-C-1-W_1 - cdf
PLOTTED: DIRECT LEMON-C-1-W_1 - tsp
TREE LEMON-C-1-W_1 | YLIM 73.24984449512505
PLOTTED: DIRECT LEMON-A-1-W_1 - cdf
PLOTTED: DIRECT LEMON-A-1-W_1 - tsp
PLOTTED: DIRECT LEMON-B-1-W_1 - cdf
PLOTTED: DIRECT LEMON-B-1-W_1 - tsp
PLOTTED: DIRECT LEMON-C-1-W_1 - cdf
PLOTTED: DIRECT LEMON-C-1-W_1 - tsp


In [68]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS       = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=2100)
core.ARGS2      = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=1900, factor=0.8, tbfont=10)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join(DIRPATH, "test")
if not os.path.isdir(dir):
    os.mkdir(dir) 
    
file="BEST-HeuristicxCOMPARISONS.png"
ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_5" },
    {"key": "heuristic", "name": "BEST",            "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotGridComparison(runs=runs, 
                        rdir=dir, 
                        title="DIRECT COMPARISONS - BEST HEURISTIC", 
                        file=file)  

file="LEMONxCOMPARISONS.png"
ARR = [
    {"key": "NONE", "name": "LEMON-1",            "root": "" },
    {"key": "NONE", "name": "LEMON-2",            "root": "" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotGridComparison(runs=runs, 
                        rdir=dir, 
                        title="DIRECT COMPARISONS - LEMONS", 
                        file=file) 

file="BEST-HeuristicxLEMONxCOMPARISONS.png"
ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_5" },
    {"key": "NONE", "name": "LEMON-2",            "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotGridComparison(runs=runs, 
                        rdir=dir, 
                        title="DIRECT COMPARISONS - BEST HEURISTIC x LEMON", 
                        file=file) 

PLOTTED: BEST-HeuristicxCOMPARISONS.png
PLOTTED: LEMONxCOMPARISONS.png
PLOTTED: BEST-HeuristicxLEMONxCOMPARISONS.png


In [35]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS       = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=2100)
core.ARGS2      = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=1900, factor=1.0, tbfont=13)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join("/home/duclos/Downloads", "finaltest")
if not os.path.isdir(dir):
    os.mkdir(dir)

file="BEST-HeuristicxW5xRANDxWORST.png"
ARR = [
     {"key": "heuristic", "name": "BEST",            "root": "W_5" },
     {"key": "p90",       "name": "WORST",          "root": "W_5" },
     {"key": "NONE",      "name": "RAND",           "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x WORST x RAND", 
                        file=file) 

file="BEST-HeuristicxW5xLEMONS.png"
ARR = [
     {"key": "heuristic", "name": "BEST",            "root": "W_5" },
     {"key": "NONE",       "name": "LEMON-1",       "root": "W_4" },
     {"key": "NONE",       "name": "LEMON-2",       "root": "W_5" },
] 
runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x LEMONS", 
                        file=file) 

    
file="BEST-HeuristicxW5xOthers.png"
ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_5" },
    {"key": "p90",       "name": "BEST",            "root": "W_5" },
    {"key": "p50",       "name": "BEST",            "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x OTHERS", 
                        file=file)  

file="BEST-HeuristicxW5xITERS.png"
d = {"key": "heuristic", "name": "BEST", "root": "W_5" } 
run = core.find(name=d["name"], key=d["key"], root=d["root"])[0]
core.plotEvalIterationsComparison(run=run, rdir=dir, title="EVALUATION COMPARISON - BEST ITERATIONS", file=file) 


file="BEST-HeuristicxW3xITERS.png"
d = {"key": "heuristic", "name": "BEST", "root": "W_3" } 
run = core.find(name=d["name"], key=d["key"], root=d["root"])[0]
core.plotEvalIterationsComparison(run=run, rdir=dir, title="EVALUATION COMPARISON - BEST ITERATIONS", file=file) 

file="BEST-HeuristicxW3xOthers.png"
ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_3" },
    {"key": "p90",       "name": "BEST",            "root": "W_3" },
    {"key": "p50",       "name": "BEST",            "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x OTHERS", 
                        file=file) 

file="BEST-HeuristicxW3xLEMONS.png"
ARR = [
     {"key": "heuristic", "name": "BEST",            "root": "W_3" },
     {"key": "NONE",       "name": "LEMON-1",       "root": "W_4" },
     {"key": "NONE",       "name": "LEMON-2",       "root": "W_5" },
] 
runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x LEMONS", 
                        file=file) 

file="BEST-HeuristicxW5xREBUILDS.png"
ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_5" },
    {"key": "heuristic", "name": "BEST-REBUILD-1",  "root": "W_5" },
    {"key": "heuristic", "name": "BEST-REBUILD-2",  "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x REBUILDS", 
                        file=file) 

file="BEST-HeuristicxW3xREBUILDS.png"
ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_3" },
    {"key": "heuristic", "name": "BEST-REBUILD-1",  "root": "W_3" },
    {"key": "heuristic", "name": "BEST-REBUILD-2",  "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x REBUILDS", 
                        file=file,fix=False) 

d = {"key": "heuristic", "name": "BEST",            "root": "W_5" }
run = core.find(name=d["name"], key=d["key"], root=d["root"])[0] 

file="BEST-HeuristicxW5xSTAGES1-3.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][:3], 
                        rdir=dir, 
                        title="BEST HEURISTIC: STAGES 1-3", 
                        file=file) 

file="BEST-HeuristicxW5xSTAGES4-7.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][3:], 
                        rdir=dir, 
                        title="BEST HEURISTIC: STAGES 4-7", 
                        file=file, base=3)

file="BEST-HeuristicxW5xSTAGESALL.png"
# core.plotStageComparison(run=run, 
#                         stages=run["stages"], 
#                         rdir=dir, 
#                         title="BEST HEURISTIC: STAGES ALL", 
#                         file=file) 

d = {"key": "p90", "name": "BEST",            "root": "W_5" }
run = core.find(name=d["name"], key=d["key"], root=d["root"])[0]  

file="BEST-p90xW5xSTAGES1-3.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][:3], 
                        rdir=dir, 
                        title="BEST p90: STAGES 1-3", 
                        file=file) 

file="BEST-p90W5xSTAGES4-7.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][3:], 
                        rdir=dir, 
                        title="BEST p90: STAGES 4-7", 
                        file=file, base=3)

d = {"key": "p50", "name": "BEST",            "root": "W_5" }
run = core.find(name=d["name"], key=d["key"], root=d["root"])[0]  

file="BEST-p50xW5xSTAGES1-3.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][:3], 
                        rdir=dir, 
                        title="BEST p50: STAGES 1-3", 
                        file=file) 

file="BEST-p50W5xSTAGES4-7.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][3:], 
                        rdir=dir, 
                        title="BEST p50: STAGES 4-7", 
                        file=file, base=3)

PLOTTED: BEST-HeuristicxW5xRANDxWORST.png
PLOTTED: BEST-HeuristicxW5xLEMONS.png
PLOTTED: BEST-HeuristicxW5xOthers.png
PLOTTED: BEST-HeuristicxW5xITERS.png
PLOTTED: BEST-HeuristicxW3xITERS.png
PLOTTED: BEST-HeuristicxW3xOthers.png
PLOTTED: BEST-HeuristicxW3xLEMONS.png
PLOTTED: BEST-HeuristicxW5xREBUILDS.png
PLOTTED: BEST-HeuristicxW3xREBUILDS.png
PLOTTED: BEST-HeuristicxW5xSTAGES1-3.png
PLOTTED: BEST-HeuristicxW5xSTAGES4-7.png
PLOTTED: BEST-p90xW5xSTAGES1-3.png
PLOTTED: BEST-p90W5xSTAGES4-7.png
PLOTTED: BEST-p50xW5xSTAGES1-3.png
PLOTTED: BEST-p50W5xSTAGES4-7.png


In [43]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS       = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=2100)
core.ARGS2      = args.Args(w=28, h=16, f=16, nf=20, tf=24, s=1900, factor=1.0, tbfont=13)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join("/home/duclos/Downloads", "test")
if not os.path.isdir(dir):
    os.mkdir(dir)

file="BEST-HeuristicxW3xRANDxWORST.png"
ARR = [
     {"key": "heuristic", "name": "BEST",            "root": "W_3" },
     {"key": "p90",       "name": "WORST",          "root": "W_3" },
     {"key": "NONE",       "name": "RAND",           "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x WORST x RAND", 
                        file=file) 

d = {"key": "heuristic", "name": "BEST",            "root": "W_3" }
run = core.find(name=d["name"], key=d["key"], root=d["root"])[0] 

file="BEST-HeuristicxW3xSTAGES1-3.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][:3], 
                        rdir=dir, 
                        title="BEST HEURISTIC: STAGES 1-3", 
                        file=file) 

file="BEST-HeuristicxW3xSTAGES4-7.png"
core.plotStageComparison(run=run, 
                        stages=run["stages"][3:], 
                        rdir=dir, 
                        title="BEST HEURISTIC: STAGES 4-7", 
                        file=file, base=3)

file="BEST-HeuristicxW3xREBUILDS.png"
ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_3" },
    {"key": "heuristic", "name": "BEST-REBUILD-1",  "root": "W_3" },
    {"key": "heuristic", "name": "BEST-REBUILD-2",  "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparison(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        title="EVALUATION COMPARISON - BEST x REBUILDS", 
                        file=file,fix=False) 

PLOTTED: BEST-HeuristicxW3xRANDxWORST.png
PLOTTED: BEST-HeuristicxW3xSTAGES1-3.png
PLOTTED: BEST-HeuristicxW3xSTAGES4-7.png
PLOTTED: BEST-HeuristicxW3xREBUILDS.png


In [70]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS2      = args.Args(w=12, h=12, f=20, nf=32, tf=35, s=1900, factor=1.0, tbfont=13)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot")
if not os.path.isdir(dir):
    os.mkdir(dir)

ARR = [
     {"key": "heuristic", "name": "BEST",           "root": "W_5" },
     {"key": "p90",       "name": "WORST",          "root": "W_5" },
     {"key": "NONE",      "name": "RAND",           "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

core.plotEvalIterationsComparisonFINAL_ALL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

ARR = [
     {"key": "heuristic", "name": "BEST",           "root": "W_5" },
     {"key": "p90",       "name": "WORST",          "root": "W_5" },
]

core.plotEvalIterationsComparisonFINAL_ALL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True)

PLOTTED: BEST-WEIGHTED-W_5 - table
PLOTTED: BEST-WEIGHTED-W_5 - cdf
PLOTTED: BEST-WEIGHTED-W_5 - p90
PLOTTED: BEST-WEIGHTED-W_5 - p50
TREE BEST-WEIGHTED-W_5 | YLIM 707.7692986897284
PLOTTED: BEST-WEIGHTED-W_5 - stddev
PLOTTED: WORST-P90-W_5 - table
PLOTTED: WORST-P90-W_5 - cdf
PLOTTED: WORST-P90-W_5 - p90
PLOTTED: WORST-P90-W_5 - p50
TREE WORST-P90-W_5 | YLIM 1337.1322402505443
PLOTTED: WORST-P90-W_5 - stddev
PLOTTED: RAND-W_5 - table
PLOTTED: RAND-W_5 - cdf
PLOTTED: RAND-W_5 - p90
PLOTTED: RAND-W_5 - p50
TREE RAND-W_5 | YLIM 863.3009822020359
PLOTTED: RAND-W_5 - stddev
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - tsp
PLOTTED: DIRECT WORST-P90-W_5 - cdf
PLOTTED: DIRECT WORST-P90-W_5 - tsp
PLOTTED: DIRECT RAND-W_5 - cdf
PLOTTED: DIRECT RAND-W_5 - tsp
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - tsp
PLOTTED: DIRECT WORST-P90-W_5 - cdf
PLOTTED: DIRECT WORST-P90-W_5 - tsp
PLOTTED: DIRECT RAND-W_5 - cdf
PLOTTED: DIRECT RAND-W_5 -

In [49]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS2      = args.Args(w=12, h=12, f=20, nf=32, tf=35, s=1900, factor=1.0, tbfont=13)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/bestW5xlemon")
if not os.path.isdir(dir):
    os.mkdir(dir)

ARR = [
       {"key": "heuristic", "name": "BEST",            "root": "W_5" },
     {"key": "NONE",       "name": "LEMON-1",       "root": "W_4" },
     {"key": "NONE",       "name": "LEMON-2",       "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/allbestxW5")
if not os.path.isdir(dir):
    os.mkdir(dir)

ARR = [
     {"key": "heuristic", "name": "BEST",            "root": "W_5" },
    {"key": "p90",       "name": "BEST",            "root": "W_5" },
    {"key": "p50",       "name": "BEST",            "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True)

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/allbestxW3")
if not os.path.isdir(dir):
    os.mkdir(dir)

ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_3" },
    {"key": "p90",       "name": "BEST",            "root": "W_3" },
    {"key": "p50",       "name": "BEST",            "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True)

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/bestW3xlemon")
if not os.path.isdir(dir):
    os.mkdir(dir)

ARR = [
     {"key": "heuristic", "name": "BEST",            "root": "W_3" },
     {"key": "NONE",       "name": "LEMON-1",       "root": "W_4" },
     {"key": "NONE",       "name": "LEMON-2",       "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True)

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/bestW5xrebuild")
if not os.path.isdir(dir):
    os.mkdir(dir)

ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_5" },
    {"key": "heuristic", "name": "BEST-REBUILD-1",  "root": "W_5" },
    {"key": "heuristic", "name": "BEST-REBUILD-2",  "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True)

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/bestW3xrebuild")
if not os.path.isdir(dir):
    os.mkdir(dir)

ARR = [
    {"key": "heuristic", "name": "BEST",            "root": "W_3" },
    {"key": "heuristic", "name": "BEST-REBUILD-1",  "root": "W_3" },
    {"key": "heuristic", "name": "BEST-REBUILD-2",  "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True)


PLOTTED: BEST-WEIGHTED-W_5 - table
PLOTTED: BEST-WEIGHTED-W_5 - cdf
PLOTTED: BEST-WEIGHTED-W_5 - p90
PLOTTED: BEST-WEIGHTED-W_5 - p50
TREE BEST-WEIGHTED-W_5 | YLIM 707.7692986897284
PLOTTED: BEST-WEIGHTED-W_5 - stddev
PLOTTED: LEMON-1-W_4 - table
PLOTTED: LEMON-1-W_4 - cdf
PLOTTED: LEMON-1-W_4 - p90
PLOTTED: LEMON-1-W_4 - p50
TREE LEMON-1-W_4 | YLIM 363.04164167097963
PLOTTED: LEMON-1-W_4 - stddev
PLOTTED: LEMON-2-W_5 - table
PLOTTED: LEMON-2-W_5 - cdf
PLOTTED: LEMON-2-W_5 - p90
PLOTTED: LEMON-2-W_5 - p50
TREE LEMON-2-W_5 | YLIM 197.37136554908363
PLOTTED: LEMON-2-W_5 - stddev
PLOTTED: BEST-WEIGHTED-W_5 - table
PLOTTED: BEST-WEIGHTED-W_5 - cdf
PLOTTED: BEST-WEIGHTED-W_5 - p90
PLOTTED: BEST-WEIGHTED-W_5 - p50
TREE BEST-WEIGHTED-W_5 | YLIM 707.7692986897284
PLOTTED: BEST-WEIGHTED-W_5 - stddev
PLOTTED: BEST-P90-W_5 - table
PLOTTED: BEST-P90-W_5 - cdf
PLOTTED: BEST-P90-W_5 - p90
PLOTTED: BEST-P90-W_5 - p50
TREE BEST-P90-W_5 | YLIM 523.3088162262891
PLOTTED: BEST-P90-W_5 - stddev
PLOTTED: B

In [73]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS2      = args.Args(w=12, h=12, f=20, nf=32, tf=35, s=1900, factor=1.0, tbfont=13)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/")
if not os.path.isdir(dir):
    os.mkdir(dir)


ARR = [
     {"key": "p90", "name": "WORST",            "root": "W_3" }
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True)

PLOTTED: WORST-P90-W_3 - table
PLOTTED: WORST-P90-W_3 - cdf
PLOTTED: WORST-P90-W_3 - p90
PLOTTED: WORST-P90-W_3 - p50
TREE WORST-P90-W_3 | YLIM 753.6949850996688
PLOTTED: WORST-P90-W_3 - stddev


In [71]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS2      = args.Args(w=12, h=12, f=20, nf=32, tf=35, s=1900, factor=1.0, tbfont=13)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/compare")
if not os.path.isdir(dir):
    os.mkdir(dir)

ARR = [
     {"key": "heuristic", "name": "BEST",           "root": "W_5" },
     {"key": "heuristic", "name": "BEST",           "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

core.plotEvalIterationsComparisonFINAL_ALL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 


ARR = [
     {"key": "p90", "name": "BEST",           "root": "W_5" },
     {"key": "p90", "name": "BEST",           "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

core.plotEvalIterationsComparisonFINAL_ALL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 


ARR = [
     {"key": "p50", "name": "BEST",           "root": "W_5" },
     {"key": "p50", "name": "BEST",           "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalComparisonFINAL(run=runs[0], 
                        runs=runs[1:], 
                        rdir=dir, 
                        suptitle=False, 
                        title=True)
core.plotEvalIterationsComparisonFINAL_ALL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 



PLOTTED: BEST-WEIGHTED-W_5 - table
PLOTTED: BEST-WEIGHTED-W_5 - cdf
PLOTTED: BEST-WEIGHTED-W_5 - p90
PLOTTED: BEST-WEIGHTED-W_5 - p50
TREE BEST-WEIGHTED-W_5 | YLIM 707.7692986897284
PLOTTED: BEST-WEIGHTED-W_5 - stddev
PLOTTED: BEST-WEIGHTED-W_3 - table
PLOTTED: BEST-WEIGHTED-W_3 - cdf
PLOTTED: BEST-WEIGHTED-W_3 - p90
PLOTTED: BEST-WEIGHTED-W_3 - p50
TREE BEST-WEIGHTED-W_3 | YLIM 485.1570168505862
PLOTTED: BEST-WEIGHTED-W_3 - stddev
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - tsp
PLOTTED: DIRECT BEST-WEIGHTED-W_3 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_3 - tsp
PLOTTED: BEST-P90-W_5 - table
PLOTTED: BEST-P90-W_5 - cdf
PLOTTED: BEST-P90-W_5 - p90
PLOTTED: BEST-P90-W_5 - p50
TREE BEST-P90-W_5 | YLIM 523.3088162262891
PLOTTED: BEST-P90-W_5 - stddev
PLOTTED: BEST-P90-W_3 - table
PLOTTED: BEST-P90-W_3 - cdf
PLOTTED: BEST-P90-W_3 - p90
PLOTTED: BEST-P90-W_3 - p50
TREE BEST-P90-W_3 | YLIM 738.248678980999
PLOTTED: BEST-P90-W_3 - stddev
PLOTTED: DIRECT BEST-P90-W_5 

In [5]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS2      = args.Args(w=12, h=12, f=20, nf=32, tf=35, s=1900, factor=1.0, tbfont=13)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/direct")
if not os.path.isdir(dir):
    os.mkdir(dir)


ARR = [
     {"key": "heuristic", "name": "BEST",           "root": "W_5" },
     {"key": "p90",       "name": "BEST",           "root": "W_5" },
     {"key": "p50",       "name": "BEST",           "root": "W_5" },
]
runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalIterationsComparisonFINAL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 


ARR = [
     {"key": "p90",       "name": "BEST",           "root": "W_5" },
     {"key": "p50",       "name": "BEST",           "root": "W_5" },
]
runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalIterationsComparisonFINAL_ALL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 


ARR = [
       {"key": "heuristic", "name": "BEST",            "root": "W_5" },
      {"key": "heuristic", "name": "BEST",            "root": "W_3" },
     {"key": "NONE",       "name": "LEMON-1",       "root": "W_4" },
     {"key": "NONE",       "name": "LEMON-2",       "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalIterationsComparisonFINAL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

core.plotEvalIterationsComparisonFINAL_ALL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

PLOTTED: DIRECT BEST-WEIGHTED-W_5 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - tsp
TREE BEST-WEIGHTED-W_5 | YLIM 183.28861778070126
PLOTTED: DIRECT BEST-P90-W_5 - cdf
PLOTTED: DIRECT BEST-P90-W_5 - tsp
TREE BEST-P90-W_5 | YLIM 521.1350335164965
PLOTTED: DIRECT BEST-P50-W_5 - cdf
PLOTTED: DIRECT BEST-P50-W_5 - tsp
TREE BEST-P50-W_5 | YLIM 103.53786070882477
PLOTTED: DIRECT BEST-P90-W_5 - cdf
PLOTTED: DIRECT BEST-P90-W_5 - tsp
PLOTTED: DIRECT BEST-P50-W_5 - cdf
PLOTTED: DIRECT BEST-P50-W_5 - tsp
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - tsp
TREE BEST-WEIGHTED-W_5 | YLIM 183.28861778070126
PLOTTED: DIRECT BEST-WEIGHTED-W_3 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_3 - tsp
TREE BEST-WEIGHTED-W_3 | YLIM 110.66919058870901
PLOTTED: DIRECT LEMON-1-W_4 - cdf
PLOTTED: DIRECT LEMON-1-W_4 - tsp
TREE LEMON-1-W_4 | YLIM 363.04164167097963
PLOTTED: DIRECT LEMON-2-W_5 - cdf
PLOTTED: DIRECT LEMON-2-W_5 - tsp
TREE LEMON-2-W_5 | YLIM 197.37136554908363
PLOTTED: DIRECT BEST-WEI

In [9]:
from analysis import core, args

core.EXPERIMENT = EXPERIMENT
core.ARGS2      = args.Args(w=12, h=12, f=20, nf=32, tf=35, s=1900, factor=1.0, tbfont=13)

matplotlib.use('agg')
plt.close("all")

dir = os.path.join("/home/duclos/Documents/work/master-arbeit/docs/images/plot/direct")
if not os.path.isdir(dir):
    os.mkdir(dir)

ARR = [
       {"key": "heuristic", "name": "BEST",            "root": "W_5" },
      {"key": "heuristic", "name": "BEST",            "root": "W_3" },
     {"key": "NONE",       "name": "LEMON-1",       "root": "W_4" },
     {"key": "NONE",       "name": "LEMON-2",       "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalIterationsComparisonFINAL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

core.plotEvalIterationsComparisonFINAL_ALL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 


ARR = [
     {"key": "heuristic", "name": "BEST",           "root": "W_5" },
     {"key": "p90",       "name": "WORST",          "root": "W_5" },
     {"key": "NONE",      "name": "RAND",           "root": "W_5" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalIterationsComparisonFINAL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

core.plotEvalIterationsComparisonFINAL_ALL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 


ARR = [
     {"key": "heuristic", "name": "BEST",           "root": "W_3" },
     {"key": "p90",       "name": "WORST",          "root": "W_3" },
     {"key": "NONE",      "name": "RAND",           "root": "W_3" },
]

runs = [ core.find(name=d["name"], key=d["key"], root=d["root"])[0] for d in ARR ]
core.plotEvalIterationsComparisonFINAL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True) 

core.plotEvalIterationsComparisonFINAL_ALL(runs=runs,
                        rdir=dir, 
                        suptitle=False, 
                        title=True)


PLOTTED: DIRECT BEST-WEIGHTED-W_5 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - tsp
TREE BEST-WEIGHTED-W_5 | YLIM 183.28861778070126
PLOTTED: DIRECT BEST-WEIGHTED-W_3 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_3 - tsp
TREE BEST-WEIGHTED-W_3 | YLIM 110.66919058870901
PLOTTED: DIRECT LEMON-1-W_4 - cdf
PLOTTED: DIRECT LEMON-1-W_4 - tsp
TREE LEMON-1-W_4 | YLIM 363.04164167097963
PLOTTED: DIRECT LEMON-2-W_5 - cdf
PLOTTED: DIRECT LEMON-2-W_5 - tsp
TREE LEMON-2-W_5 | YLIM 197.37136554908363
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - tsp
PLOTTED: DIRECT BEST-WEIGHTED-W_3 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_3 - tsp
PLOTTED: DIRECT LEMON-1-W_4 - cdf
PLOTTED: DIRECT LEMON-1-W_4 - tsp
PLOTTED: DIRECT LEMON-2-W_5 - cdf
PLOTTED: DIRECT LEMON-2-W_5 - tsp
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - cdf
PLOTTED: DIRECT BEST-WEIGHTED-W_5 - tsp
TREE BEST-WEIGHTED-W_5 | YLIM 183.28861778070126
PLOTTED: DIRECT WORST-P90-W_5 - cdf
PLOTTED: DIRECT WORST-P90-W_5 - tsp
TREE WORST-P90-W_5 | YL